# OpenAI GPT Assistances #

A simple example that demonstrates how to use OpenAI's Assistant API. It has a lot of similar functionality to the Completions and Responses API, but allows you to maintain chistory across sessions and provides the ability to ingest data files. 

In [ ]:
# pip install openai
import openai

## Creating a GPT Session ##

I previously saved my API Key to an OS environment variable so that it's not published online



In [ ]:
import os

my_api_key = os.environ.get("OPENAI_API_KEY")
print(f"Connecting to ChatGPT session using API key:\n'{my_api_key}'")
openai_session = openai.OpenAI(api_key=my_api_key)
session_running = not openai_session.is_closed()
print(f"Session established: {session_running}")

## Uploading Custom Files ##

Users can upload files to GPT Assistants that allow GPT to search their documents for answers. This is not the same as training the AI: training involves changing the weights of the underlying neural network. Instead, these files simply provide non-public information for GPT to search.

Files are uploaded to OpenAI and reside in a storage area that is associated with the given API key. By default, these files are private and can only be accessed by the GPT assistant associated with the API key. Users are charged on a per-file, per-GB, per-day basis regardless of whether their assistant is active. But the fees are relatively small, on the order of 20 cents per GB per day.

To avoid being overcharged, it is important to avoid uploading the same file multiple times. Check to see if the file exists in the GPT storage area before uploading another copy. This code will verify by filename only; advanced users might compare file hashes.

Once a file is uploaded into the OpenAI storage area, reference it with its ID.

In [ ]:
print(f"List of files associated with my OpenAI account: ")
remote_files = openai_session.files.list().data
for remote_file in remote_files:
    print(f" -> {remote_file.filename} (ID {remote_file.id})")

In [ ]:
local_filename = "internship_checklist.txt"
choice = input(f"Are you sure that you want to upload the file {local_filename}? ")
if choice.lower().startswith('y'):
    remote_file = openai_session.files.create(
        file=open(local_filename, "rb"),
        purpose="assistants"  # this is a magic value for GPT context documents
    )
    # purpose='assistants'
    # purpose='batch'
    # purpose='fine-tune'
    # purpose='vision'
    # purpose='user_data'
    # purpose='evals'
    print(f"Uploaded file {local_filename} (ID {remote_file.id})")

## Create the Assistant ##

We will create an Assistant that is tied to our OpenAI API key. It will not be publically available unless we specifically configure it that way.

Assistants are similar to Files in that they persist between sessions. So you will want to double-check whether a particular assistant exists before bothering to create a new one.

In [ ]:
print(f"List of assistants associated with my OpenAI account:")
saved_assistants = openai_session.beta.assistants.list()
for assistant in saved_assistants.data:
    print(f" -> {assistant.name} (ID {assistant.id}): {assistant.description}")

In [ ]:
#openai.beta.assistants.delete("asst_tgjOhrgM3AQ2PArlQ7tMSa6G")

In [ ]:
gpt_name = "Marty"
gpt_description = "Prof. Tallman's Assistant"
gpt_context = (
    "Marty is the mascot for Concordia Univeristy Irvine, a golden eagle. He is always "
    "positive, enthusiastic, and kind. In addition to flying, he likes to surf and hang "
    "out with students. He is a wise, honerable, and cultivated Christian citizen."
)
gpt_model = "gpt-4o-mini"

choice = input(f"Are you sure that you want to create a new assisstant named {gpt_name}? ")
if choice.lower().startswith('y'):
    assistant = openai_session.beta.assistants.create(
        name=gpt_name,
        description=gpt_description,
        instructions=gpt_context,
        model=gpt_model,
        tools=[{"type": "file_search"}] # to work with documents
    )
    # {"type": "function"}
    # {"type": "file_search"}
    # {"type": "code_interpreter"}
    print(f"Created AI Assistant '{assistant.name}' ({assistant.instructions[:50]}...)")

else:
    assistant = saved_assistants[-1]

## Create a Discussion Thread ##

Previously, with the simpler AI, we had to manually set the context and conversation history with every request. The Assistants API is a little easier to work with because GPT will internally keep track of these things so that all you need to do is send each prompt and read the response.

### Thread Persistence ###

Threads survive across OpenAI sessions, similar to files and assistants. However, the main difference is that there is no way to list or search for existing threads. You are responsible for tracking all of your own threads by thread ID. This is important to for restoring previous discussions and avoiding clutter within the user's account (admittidely, the impact of lost threads is negligible... it just annoys those with OCD tendencies). 

In [ ]:
thread = openai_session.beta.threads.create()
print(f"Created a new discussion thread '{thread.id}'")
print(f"Either save this value to restore the conversation, or delete it when finished!")

## Running the Conversation ##


In [ ]:
import time

# Try-except-finally helps us to always delete the thread when finished
try:
    user_msg = input("\n> ")

    print(f"Sending message to GPT...")
    attachments = [ { 'file_id':remote_file.id, 'tools':[{'type': 'file_search'}] } ]
    # 'type': 'file_search'
    # 'type': 'code_interpreter'
    prompt = openai_session.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_msg,
        attachments=attachments
    )
    response = openai_session.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    print(f"Waiting for GPT response...")
    while response.status != "completed":
        time.sleep(1) # avoid checking too often
        response = openai_session.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=response.id
        )

    # Retrieve and display the assistant's last response
    messages = openai_session.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data:
        if message.role == "assistant":
            print(f"\nAssistant: {message.content[0].text.value}")
            break # display the most recent response and then stop

finally:
    openai_session.beta.threads.delete(thread_id=thread.id)
    print(f"\nThread '{thread.id}' has been deleted")